<a href="https://colab.research.google.com/github/itsadi1/Agentic-AI/blob/main/Agenticai(ClimateChange).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  !pip install ibm_watsonx_ai
#  !pip install langchain_ibm

In [2]:
from ibm_watsonx_ai import APIClient,Credentials
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from google.colab import userdata
import textwrap, json, re

In [3]:
# Initialize credentials and API client
credentials = Credentials(
    url=userdata.get('ibm_cloud_url'),
    api_key=userdata.get('watson_api_key')
)
project_id = userdata.get('ibm_project_id')
client = APIClient(credentials)

In [4]:
api_client = APIClient(credentials=credentials, project_id=project_id)

In [5]:
parameters = {
    # GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,  # Greedy for deterministic output
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 100,
    GenParams.TEMPERATURE: 0.85,
    GenParams.TOP_K: 20,
    GenParams.TOP_P: 0.6
    }

In [6]:
api_client.foundation_models.TextModels.show()

{'FLAN_T5_XL': 'google/flan-t5-xl', 'GRANITE_13B_INSTRUCT_V2': 'ibm/granite-13b-instruct-v2', 'GRANITE_3_2_8B_INSTRUCT': 'ibm/granite-3-2-8b-instruct', 'GRANITE_3_2B_INSTRUCT': 'ibm/granite-3-2b-instruct', 'GRANITE_3_3_8B_INSTRUCT': 'ibm/granite-3-3-8b-instruct', 'GRANITE_3_8B_INSTRUCT': 'ibm/granite-3-8b-instruct', 'GRANITE_8B_CODE_INSTRUCT': 'ibm/granite-8b-code-instruct', 'GRANITE_GUARDIAN_3_2B': 'ibm/granite-guardian-3-2b', 'GRANITE_GUARDIAN_3_8B': 'ibm/granite-guardian-3-8b', 'GRANITE_VISION_3_2_2B': 'ibm/granite-vision-3-2-2b', 'LLAMA_2_13B_CHAT': 'meta-llama/llama-2-13b-chat', 'LLAMA_3_2_11B_VISION_INSTRUCT': 'meta-llama/llama-3-2-11b-vision-instruct', 'LLAMA_3_2_1B_INSTRUCT': 'meta-llama/llama-3-2-1b-instruct', 'LLAMA_3_2_3B_INSTRUCT': 'meta-llama/llama-3-2-3b-instruct', 'LLAMA_3_2_90B_VISION_INSTRUCT': 'meta-llama/llama-3-2-90b-vision-instruct', 'LLAMA_3_3_70B_INSTRUCT': 'meta-llama/llama-3-3-70b-instruct', 'LLAMA_3_405B_INSTRUCT': 'meta-llama/llama-3-405b-instruct', 'LLAMA_4_

In [7]:
# Initialize LLM
llm = WatsonxLLM(
    model_id="meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    url=credentials["url"],
    apikey=credentials["api_key"],
    project_id=project_id,
    params=parameters
)

In [8]:
#Wrapping Content
def printable(base):
  if type(base)==dict:
    for role, content in base.items():
        content_str = json.dumps(content) if not isinstance(content, str) else content
        wrapped_content = textwrap.fill(content_str, width=80, initial_indent="  ", subsequent_indent="  ")
        print(f"{role.capitalize()} Agent:\n{wrapped_content}\n")
  elif type(base)==str:
    wrapped_content = textwrap.fill(base, width=80, initial_indent="  ", subsequent_indent="  ")
    print(wrapped_content)
  else:
    print("Error")

In [9]:
def process_query(role: str, query: str, llm) -> str:
    # Define prompt template for JSON output using 'role' and 'question'
    prompt = PromptTemplate(
        input_variables=["role", "question"],
        template="""
Given the role "{role}" and the question "{question}", provide the answer in JSON format as follows:
```json
{{
    "role": "{role}",
    "question": "{question}",
    "answer": "<your concise answer in one sentence, without restating the question or adding explanations>",
    "verdict": "[negative,neutral,positive]"
}}
Just provide the Final Answer directly in JSON format, ensuring the response is valid JSON.
"""
    )

    # Format the prompt using role and query (question)
    formatted_prompt = prompt.format(role=role, question=query)

    # Use the LLM to generate the response
    raw_response = llm.invoke(formatted_prompt)
    st=raw_response.index('{')
    en=raw_response.index('}')
    return json.loads(raw_response[st:en+1])


In [10]:
#Agent: Env
class EnvAgent:
  def __init__(self,llm):
    self.role = "Act as an environment expert keeping up with the latest climate data from IPCC, NASA, and global monitoring systems."
    self.task = "Provide climate data and environmental insights from IPCC, NASA, and global monitoring systems."
    self.llm = llm
  def run(self):
    return  process_query(self.role, self.task, self.llm).get("answer")

  def query(self, userquery):
    return process_query(self.role, userquery, self.llm)

  def get_role(self):
    return "env"

In [11]:
# Agent: Tech
class TechAgent:
    def __init__(self, llm):
        self.role = "Act as a technology and economics expert analyzing the feasibility and costs of climate mitigation technologies."
        self.task = "Provide technology and economic feasibility data for climate mitigation solutions."
        self.llm = llm

    def run(self):
        return process_query(self.role, self.task, self.llm).get("answer")

    def query(self, userquery):
        return process_query(self.role, userquery, self.llm)

    def get_role(self):
        return "tech"

In [12]:
# Agent: Social
class SocialAgent:
    def __init__(self, llm):
        self.role = "Act as a social scientist analyzing public sentiment and social impacts of climate policies using surveys and online data."
        self.task = "Provide public sentiment and social impact data for climate policies."
        self.llm = llm

    def run(self):
        return process_query(self.role, self.task, self.llm).get("answer")

    def query(self, userquery):
        return process_query(self.role, userquery, self.llm)

    def get_role(self):
        return "social"

In [13]:
# Agent: Policy
class PolicyAgent:
    def __init__(self, llm):
        self.role = "Act as a policy analyst reviewing global and national climate policies from UNFCCC, government portals, and NGOs."
        self.task = "Provide global and national climate policy data and alignment analysis."
        self.llm = llm

    def run(self):
        return process_query(self.role, self.task, self.llm).get("answer")

    def query(self, userquery):
        return process_query(self.role, userquery, self.llm)

    def get_role(self):
        return "policy"

In [14]:
class CentralAI:
  def __init__(self):
    self.role="Act as the head of a climate policy department, coordinating and synthesizing insights from technology, social, policy, and environmental teams via attached comprehensive knowledge base to develop evidence-based climate strategies."
    self.kb = {}
    self.tech = TechAgent(llm)
    self.policy = PolicyAgent(llm)
    self.social = SocialAgent(llm)
    self.env = EnvAgent(llm)
    self.agents = [self.tech, self.policy, self.social, self.env]

  def create_kb(self):
    self.kb["tech"] = self.tech.run()
    self.kb["policy"] = self.policy.run()
    self.kb["social"] = self.social.run()
    self.kb["env"] = self.env.run()

  def research(self) -> str:
    if self.kb == {}:
      self.create_kb()
    # Define prompt template for JSON output using 'role' and 'question'

    prompt = PromptTemplate(
        input_variables=["role", "knowledge_base"],
        template="""
Given the role "{role}" and the complete knowledge base "{knowledge_base}", provide the answer in JSON format as follows:
```json
{{
    "prediction": "<your concise predicted practice/policy/law in one sentence creating impact towards the goal of climate actions, without restating the question or adding explanations>",
}}
Just provide the Final Answer directly in JSON format, ensuring the response is valid JSON.
"""
    )

    # Format the prompt using role and query (question)
    formatted_prompt = prompt.format(role=self.role, knowledge_base=self.kb)

    # Use the LLM to generate the response
    raw_response = llm.invoke(formatted_prompt)
    st=raw_response.index('{')
    en=raw_response.index('}')
    return json.loads(raw_response[st:en+1])

  def process(self, userquery, max_recursions =3):
    counter=[]
    prefix = "Based on your role, make an unbaised judgement on the provided statement: "
    for agent in self.agents:
      reply = agent.query(prefix+userquery)
      if reply.get("verdict") == "negative":
        self.kb[agent.get_role()] += "\n"+reply.get("answer")
        counter.append(False)
      else:
        counter.append(True)
    while not all(counter) and max_recursions > 0:
      userquery = self.research().get("prediction")
      return self.process(userquery, max_recursions=max_recursions-1)

    return userquery, counter

In [15]:
if __name__ =="__main__":
  cai = CentralAI()
  initial_prediction = cai.research().get("prediction")
  final_prediction,verdicts=cai.process(initial_prediction)
  format_verdict = lambda x: "positive" if x else "negative"
  verdict = list(map(format_verdict,verdicts))
  print("Knowledge Base populated by Multiple Specialized AI Agents: ")
  printable(cai.kb)
  print("Final Prediction:")
  printable(final_prediction)
  print("Agents Validation:",verdict)

Knowledge Base populated by Multiple Specialized AI Agents: 
Tech Agent:
  Renewable energy technologies like solar and wind have become increasingly
  cost-competitive with fossil fuels, with levelized costs ranging from $30 to
  $50 per MWh.

Policy Agent:
  Global climate policies, such as the Paris Agreement, are aligned with
  national policies, like the US Climate Action Plan, in reducing greenhouse gas
  emissions and promoting renewable energy.

Social Agent:
  Our analysis indicates a generally positive public sentiment towards climate
  policies, with 62% of surveyed individuals supporting stricter emission
  controls.

Env Agent:
  The latest climate data indicates a global temperature rise of 1.1°C above
  pre-industrial levels, with severe implications for ecosystems and human
  societies.

Final Prediction:
  Implement a global renewable energy mandate with emission reduction targets.
Agents Validation: ['positive', 'positive', 'positive', 'positive']
